<a href="https://colab.research.google.com/github/levflevine/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:15 http://pp

In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

### Create DataFrames to match tables

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y|
|R2HBOEM8LE9928|          5|            0|          0|   N|                Y|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y|
|R1EBPM82ENI67M|          1|            0|          0|   N|                Y|
|R372S58V6D11AT|          5|            1|          1|   N|                Y|
|R1A4514XOYI1PD|          5|            1|          1|   N|                Y|
|R20D9EHB7N20V6|          5|            0|          0|   N|                Y|
|R1WUTD8MVSROJU|          5|            0|          0|   N|                Y|
|R1QCYLT25812DM|          4|            0|          0|   N|                Y|
| R904DQPBCEM7A|          4|            0|          0|   N|     

In [6]:
# 1. Retrieve all the rows where the total_votes count is equal to or greater than 20 
twenty_votes_df = vine_df.filter(vine_df.total_votes>=20)
twenty_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [7]:
# 2. Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_votes_df = twenty_votes_df.filter(twenty_votes_df.helpful_votes/twenty_votes_df.total_votes>=0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [12]:
# 3. Retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
review_paid_df = helpful_votes_df.filter(helpful_votes_df.vine=="Y")
review_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N|
| R82QWN2X2OCHB|          5|          176|        208|   Y|                N|
|R1UYHBYE6790BU|          5|           44|         53|   Y|                N|
|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N|
|R3QDI539WTXKE2|          5|           26|         32|   Y|                N|
| RQTPWY6ND2NRV|          4|           37|         37|   Y|                N|
| RQZSTE0KOBY2G|          4|           98|        109|   Y|                N|
| RF2RNZEJO0K1N|          4|           23|         26|   Y|                N|
| ROB6VOW41E8O5|          4|          155|        172|   Y|                N|
|R3ASMXPEPYY28T|          3|           11|         20|   Y|     

In [13]:
# 4. Retrieves all the rows where a review was written, vine == 'N'
review_free_df = helpful_votes_df.filter(helpful_votes_df.vine=="N")
review_free_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1FBO737KD9F2N|          5|           19|         23|   N|                Y|
|R227GSNWI6BSZV|          1|           20|         20|   N|                Y|
|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y|
|R248FG65D76D5Y|          1|           42|         53|   N|                Y|
|R3B6BXFKGW52SG|          1|           32|         32|   N|                Y|
| ROTIV4VYL31R4|          5|           26|         26|   N|                Y|
|R3VQ59LD2LSKY7|          5|           20|         25|   N|                Y|
| RIIGLD8JB7PX8|          1|           32|         35|   N|                Y|
|R3MUBV21QV0IJK|          3|           77|         84|   N|                Y|
|R1V5W0X6BKIJYA|          5|          129|        132|   N|     

In [22]:
# 5.Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
total_reviews_paid = review_paid_df.count()
total_reviews_free = review_free_df.count()

five_star_reviews_paid = review_paid_df.filter(review_paid_df.star_rating==5).count()
five_star_reviews_free = review_free_df.filter(review_free_df.star_rating==5).count()

percent_five_star_paid = five_star_reviews_paid / total_reviews_paid * 100
percent_five_star_free = five_star_reviews_free / total_reviews_free * 100

print(f"Total Number of Vine Program Reviews (paid): {total_reviews_paid:,}\n" 
    f"Total Number of Reviews that are not part of Vine Program: {total_reviews_free:,}\n"
    f"Number of Five Start Reviews that are part of Vine Program: {five_star_reviews_paid:,}\n"
    f"Number of Five Start Reviews that are not part of Vine Program: {five_star_reviews_free:,}\n"
    f"Percent of Five Start Reviews that are part of Vine Program: {int(percent_five_star_paid)}%\n"
    f"Percent of Five Start Reviews that are not part of Vine Program: {int(percent_five_star_free)}%\n"
)


Total Number of Vine Program Reviews (paid): 1,080
Total Number of Reviews that are not part of Vine Program: 49,673
Number of Five Start Reviews that are part of Vine Program: 454
Number of Five Start Reviews that are not part of Vine Program: 23,043
Percent of Five Start Reviews that are part of Vine Program: 42%
Percent of Five Start Reviews that are not part of Vine Program: 46%



In [34]:
# Summary Statistics for Reviews that are part of Vine Program
paid_ratings = review_paid_df.select(review_paid_df.star_rating)
paid_ratings.summary().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|              1080|
|   mean| 4.093518518518518|
| stddev|0.9834377437155114|
|    min|                 1|
|    25%|                 4|
|    50%|                 4|
|    75%|                 5|
|    max|                 5|
+-------+------------------+



In [35]:
# Summary Statistics for Reviews that are not part of Vine Program
free_ratings = review_free_df.select(review_free_df.star_rating)
free_ratings.summary().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|             49673|
|   mean|3.6498500191250782|
| stddev|1.5646972640859573|
|    min|                 1|
|    25%|                 2|
|    50%|                 4|
|    75%|                 5|
|    max|                 5|
+-------+------------------+



In [47]:
# 2-Sample Statistical Test

import pandas as pd
from statsmodels.stats.weightstats import ttest_ind
fr = free_ratings.select('star_rating').rdd.flatMap(lambda x: x).collect()
pr = paid_ratings.select('star_rating').rdd.flatMap(lambda x: x).collect()
ttest_ind(fr, pr)

(-9.278559374882395, 1.7831621475444042e-20, 50751.0)